In [9]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -v -p numpy,sklearn,pandas
%watermark -v -p cv2,PIL,matplotlib
%watermark -v -p torch,torchvision,torchaudio
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

from IPython.display import display, Markdown, HTML, Javascript
display(HTML('<style>.container { width:%d%% !important; }</style>' % 90))

def _IMPORT_(x):
    try:
        exec(x, globals())
    except:
        pass


CPython 3.6.9
IPython 7.16.1

numpy 1.19.4
sklearn 0.24.0
pandas 1.1.5
CPython 3.6.9
IPython 7.16.1

cv2 4.5.1
PIL 6.2.2
matplotlib 3.3.3
CPython 3.6.9
IPython 7.16.1

torch 1.8.0.dev20210103+cu101
torchvision 0.9.0.dev20210103+cu101
torchaudio not installed


In [10]:
###
### Common ###
###

import sys, os, io, time, random, math
import json, base64, requests
import os.path as osp

_IMPORT_('import numpy as np')
_IMPORT_('import pandas as pd')
_IMPORT_('from tqdm.notebook import tqdm')

def print_progress_bar(x):
    print('\r', end='')
    print('Progress: {}%:'.format(x), '%s%s' % ('▋'*(x//2), '.'*((100-x)//2)), end='')
    sys.stdout.flush()



###
### Display ###
###

_IMPORT_('import cv2')
_IMPORT_('from PIL import Image')
_IMPORT_('from torchvision.utils import make_grid')
_IMPORT_('import matplotlib.pyplot as plt')
_IMPORT_('import plotly')
_IMPORT_('import plotly.graph_objects as go')

# plotly.offline.init_notebook_mode(connected=False)

plt.rcParams['figure.figsize'] = (12.0, 8.0)

def show_table(headers, data, width=900):
    ncols = len(headers)
    width = int(width / ncols)
    lralign = []
    caption = []
    for item in headers:
        astr = ''
        if item[0] == ':':
            astr = ':'
            item = item[1:]
        astr += '---'
        if item[-1] == ':':
            astr += ':'
            item = item[:-1]
        lralign.append(astr)
        caption.append(item)
    captionstr = '|'.join(caption) + chr(10)
    lralignstr = '|'.join(lralign) + chr(10)
    imgholdstr = '|'.join(['<img width=%d/>' % width] * ncols) + chr(10)
    table = captionstr + lralignstr + imgholdstr
    is_dict = isinstance(data[0], dict)
    for row in data:
        if is_dict:
            table += '|'.join([f'{row[c]}' for c in caption]) + chr(10)
        else:
            table += '|'.join([f'{col}' for col in row]) + chr(10)
    return Markdown(table)

def show_video(vidsrc, width=None, height=None):
    W, H = '', ''
    if width:
        W = 'width=%d' % width
    if height:
        H = 'height=%d' % height
    if vidsrc.startswith('http'):
        data_url = vidsrc
    else:
        mp4 = open(vidsrc, 'rb').read()
        data_url = 'data:video/mp4;base64,' + base64.b64encode(mp4).decode()
    return HTML('<video %s %s controls src="%s" type="video/mp4"/>' % (W, H, data_url))

def show_image(imgsrc, width=None, height=None):
    if isinstance(imgsrc, np.ndarray):
        img = imgsrc
        if width or height:
            if width and height:
                size = (width, height)
            else:
                rate = img.shape[1] / img.shape[0]
                if width:
                    size = (width, int(width/rate))
                else:
                    size = (int(height*rate), height)
            img = cv2.resize(img, size)
            plt.figure(figsize=(3*int(size[0]/80+1), 3*int(size[1]/80+1)), dpi=80)
        plt.axis('off')
        if len(img.shape) > 2:
            plt.imshow(img);
        else:
            plt.imshow(img, cmap='gray');
        return

    W, H = '', ''
    if width:
        W = 'width=%d' % width
    if height:
        H = 'height=%d' % height
    if imgsrc.startswith('http'):
        data_url = imgsrc
    else:
        if len(imgsrc) > 2048:
            data_url = 'data:image/jpg;base64,' + imgsrc
        else:
            img = open(imgsrc, 'rb').read()
            data_url = 'data:image/jpg;base64,' + base64.b64encode(img).decode()
    return HTML('<img %s %s src="%s"/>' % (W, H, data_url))

    W, H = '', ''
    if width:
        W = 'width=%d' % width
    if height:
        H = 'height=%d' % height
    if image_path.startswith('http'):
        data_url = image_path
    else:
        img = open(image_path, 'rb').read()
        data_url = 'data:image/jpg;base64,' + base64.b64encode(img).decode()
    return HTML('<img %s %s src="%s"/>' % (W, H, data_url))

def im_read(url, rgb=True, size=None):
    response = requests.get(url)
    if response:
        imgmat = np.frombuffer(response.content, dtype=np.uint8)
        img = cv2.imdecode(imgmat, cv2.IMREAD_COLOR)
        if rgb:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if size:
           if isinstance(size, int):
               size = (size, size)
           img = cv2.resize(img, size, interpolation=cv2.INTER_AREA)
        return img
    return None


In [3]:
import os
import csv
import json
import urllib
import requests
import random
import base64

from pprint import pprint
from random import sample
from urllib.request import urlretrieve
from k12libs.utils.nb_easy import k12ai_get_top_dir, RACEURL
from k12libs.utils.nb_easy import K12AI_DATASETS_ROOT
from IPython.display import display, HTML

In [4]:
API_INFERENCE = f'{RACEURL}/raceai/framework/inference'
def img2b64(path):
    with open(path, 'rb') as fr:
        b4data = base64.b64encode(fr.read()).decode()
    return b4data

## Base64 Input:

```json
{
    "task": "plate_color.inference",
    "cfg": {
        "data": {
            "class_name": "raceai.data.process.Base64DataLoader",
            "params": {
                "data_source": "b64str",
                "dataset": {
                    "class_name": "raceai.data.RawRaceDataset",
                    "params": {}
                 }
             }
        },
        "model": {
           "class_name": "raceai.models.xfeatures.PlateColorFeatures",  
           "params": {
               "ellsize": 15,
               "iterations": 3,
               "edge_thickness": 30
           }
       }
    }
}
```


## List Path Input:

```json
{
    "task": "plate_color.inference",
    "cfg": {
        "data": {
            "class_name": "raceai.data.process.PathListDataLoader",
            "params": {
                "data_source": [{"image_id": 1, "image_path": "https://raceai.s3.didiyunapi.com/data/datasets/cv/dining_plate/blue.png"}],
                "dataset": {
                    "class_name": "raceai.data.PredictListImageRaw",
                    "params": {}
                 }
             }
        },
        "model": {
           "class_name": "raceai.models.xfeatures.PlateColorFeatures",  
           "params": {
               "ellsize": 15,
               "iterations": 3,
               "edge_thickness": 30
           }
       }
    }
}
```

In [1]:
reqdata = '''{
    "task": "plate_color.inference",
    "cfg": {
        "data": {
            "class_name": "raceai.data.process.PathListDataLoader",
            "params": {
                "data_source": [
                    {"image_id": 1, "image_path": "green/green1.jpg"},
                    {"image_id": 2, "image_path": "green/green2.jpg"},
                    {"image_id": 3, "image_path": "green/green3.jpg"},
                    {"image_id": 4, "image_path": "green/green4.jpg"},
                    {"image_id": 5, "image_path": "green/green5.jpg"},
                    {"image_id": 6, "image_path": "blue/blue1.jpg"},
                    {"image_id": 7, "image_path": "blue/blue2.jpg"},
                    {"image_id": 8, "image_path": "blue/blue4.jpg"},
                    {"image_id": 9, "image_path": "blue/blue5.jpg"},
                    {"image_id": 12, "image_path": "orange/orange1.jpg"},
                    {"image_id": 13, "image_path": "orange/orange2.jpg"},
                    {"image_id": 11, "image_path": "orange/orange3.jpg"},
                    {"image_id": 14, "image_path": "orange/orange4.jpg"},
                    {"image_id": 10, "image_path": "orange/orange5.jpg"},
                    {"image_id": 18, "image_path": "red/red1.jpg"},
                    {"image_id": 19, "image_path": "red/red2.jpg"},
                    {"image_id": 20, "image_path": "red/red3.jpg"},
                    {"image_id": 21, "image_path": "red/red4.jpg"},
                    {"image_id": 21, "image_path": "yellow/yellow1.jpg"},
                    {"image_id": 22, "image_path": "yellow/yellow2.jpg"},
                    {"image_id": 23, "image_path": "yellow/yellow3.jpg"},
                    {"image_id": 24, "image_path": "yellow/yellow4.jpg"},
                    ],
                "dataset": {
                    "class_name": "raceai.data.PredictListImageRaw",
                    "params": {
                        "data_prefix": "https://raceai.s3.didiyunapi.com/data/datasets/cv/Dinner_plate/"
                    }
                 }
             }
        },
        "model": {
           "class_name": "raceai.models.xfeatures.PlateColorFeatures",  
           "params": {
               "ellsize": 8,
               "iterations": 3,
               "edge_thickness": 10
           }
       }
    }
}'''
# }''' % (img2b64('/data/nb_data/cv/plate_color/blue.png'))

print(reqdata)

{
    "task": "plate_color.inference",
    "cfg": {
        "data": {
            "class_name": "raceai.data.process.PathListDataLoader",
            "params": {
                "data_source": [
                    {"image_id": 1, "image_path": "green/green1.jpg"},
                    {"image_id": 2, "image_path": "green/green2.jpg"},
                    {"image_id": 3, "image_path": "green/green3.jpg"},
                    {"image_id": 4, "image_path": "green/green4.jpg"},
                    {"image_id": 5, "image_path": "green/green5.jpg"},
                    {"image_id": 6, "image_path": "blue/blue1.jpg"},
                    {"image_id": 7, "image_path": "blue/blue2.jpg"},
                    {"image_id": 8, "image_path": "blue/blue4.jpg"},
                    {"image_id": 9, "image_path": "blue/blue5.jpg"},
                    {"image_id": 12, "image_path": "orange/orange1.jpg"},
                    {"image_id": 13, "image_path": "orange/orange2.jpg"},
                    {"imag

## Output:

```

colors_names = ['black', 'gray', 'white', 'red', 'orange', 'yellow', 'green', 'cyan', 'blue', 'purple']


{
    "errno": 0,
    "result": [
        {
            "argmax": 8,                // 颜色数组的的下标, 8:blue
            "image_id": 1,              // 传递过来的图标id
            "image_path": "/tmp/tmpnfjfhwdh/blue.png",  // 本地调试使用
            "probs": [                                  // 概率数组, 概率最大的下标就是最终答案.
                0.0023584905660377358,
                0.0,
                0.030660377358490566,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.9669811320754716,
                0.0
            ]
        }
    ]
}
```

In [25]:
cfg = eval(reqdata)
resdata = json.loads(requests.post(url=API_INFERENCE, json=cfg).text)
print(json.dumps(resdata, indent=4))

{
    "errno": 0,
    "result": [
        {
            "argmax": 6,
            "image_id": 1,
            "image_path": "/tmp/tmpnlk9cu6x/green1.jpg",
            "probs": [
                0.0015060240963855422,
                0.01957831325301205,
                0.030120481927710843,
                0.0,
                0.0,
                0.25,
                0.6987951807228916,
                0.0,
                0.0,
                0.0
            ]
        },
        {
            "argmax": 4,
            "image_id": 2,
            "image_path": "/tmp/tmpnlk9cu6x/green2.jpg",
            "probs": [
                0.016605166051660517,
                0.02952029520295203,
                0.01845018450184502,
                0.0,
                0.8597785977859779,
                0.06088560885608856,
                0.014760147601476014,
                0.0,
                0.0,
                0.0
            ]
        },
        {
            "argmax": 6,
            "im

In [28]:
#                  0        1      2        3       4          5        6       7       8         9 
colors_names = ['black', 'gray', 'white', 'red', 'orange', 'yellow', 'green', 'cyan', 'blue', 'purple']
for item in resdata['result']:
    item['target_id'] = colors_names.index(os.path.basename(item['image_path'][:-5]))

In [29]:
show_table([':image_path', ':argmax:', 'target_id'], resdata['result'])

image_path|argmax|target_id
:---|:---:|---
<img width=300/>|<img width=300/>|<img width=300/>
/tmp/tmpnlk9cu6x/green1.jpg|6|6
/tmp/tmpnlk9cu6x/green2.jpg|4|6
/tmp/tmpnlk9cu6x/green3.jpg|6|6
/tmp/tmpnlk9cu6x/green4.jpg|6|6
/tmp/tmpnlk9cu6x/green5.jpg|6|6
/tmp/tmpnlk9cu6x/blue1.jpg|8|8
/tmp/tmpnlk9cu6x/blue2.jpg|8|8
/tmp/tmpnlk9cu6x/blue4.jpg|8|8
/tmp/tmpnlk9cu6x/blue5.jpg|8|8
/tmp/tmpnlk9cu6x/orange1.jpg|4|4
/tmp/tmpnlk9cu6x/orange2.jpg|2|4
/tmp/tmpnlk9cu6x/orange3.jpg|3|4
/tmp/tmpnlk9cu6x/orange4.jpg|4|4
/tmp/tmpnlk9cu6x/orange5.jpg|3|4
/tmp/tmpnlk9cu6x/red1.jpg|1|3
/tmp/tmpnlk9cu6x/red2.jpg|3|3
/tmp/tmpnlk9cu6x/red3.jpg|3|3
/tmp/tmpnlk9cu6x/red4.jpg|2|3
/tmp/tmpnlk9cu6x/yellow1.jpg|2|5
/tmp/tmpnlk9cu6x/yellow2.jpg|4|5
/tmp/tmpnlk9cu6x/yellow3.jpg|4|5
/tmp/tmpnlk9cu6x/yellow4.jpg|1|5
